# This is the script for the linear regression model
## The imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
#from torch.utils.tensorboard import SummaryWriter
import FinalBorealis as dg
#import datetime
#import json
#import itertools
import nni

I decided to create the module as a class so I could give it an input size and output size, and for future reusability

In [2]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
    def forward(self, x):
        return self.linear(x)

This is how the training of the model is done. For this function I used inspiration from the example model given by Borealis, this function receives:
    - Model: In this case a linear regression model.
    - Opt: Optimization function.
    - loss_fn: Loss Function.
    - train_ld: the data.
    - n_epochs (default = 3): Epochs of training.
    - save_to (default = './trainedModel.pth'): The file generated for the model

In [3]:
def train(model, opt, loss_fn, train_ld, writer, n_epoch=3, textFile='test.txt'):
    ##Iterate through a number of epochs
    for epoch in range(n_epoch):
        ##Training with batches of data
        running_loss = 0.0
        num_iters = len(train_ld)
        for i, data in enumerate(train_ld,0):
            inputs, labels = data[:,:16], data[:,16:]
            #First step: Generate predictions
           
            pred = model(inputs)
           
            #2nd step: Calculate loss
            loss = loss_fn(pred, labels)
            loss.backward()
            opt.step()

            opt.zero_grad()
            running_loss += loss.item()
            if i % 100== 0:
                #print(f"Loss at epoch [{epoch}/{n_epoch}], iteration [{i + 1}/{num_iters}]:{running_loss /2000}")
                writer.add_scalar('training loss', running_loss/100, epoch * len(train_ld) + i)
                running_loss = 0.0
    #print(f"Training done after {n_epoch} epochs!")
    with open(textFile, 'a') as f:
        f.write(f"Training done after {n_epoch} epochs!\n")
    #torch.save(model.state_dict(), save_to)
    #print(f"Trained model is saved to {save_to}.")


In [4]:
def test(testLoader, model, writer, textFile):
    #model = linearRegression(16,1)
    #model.load_state_dict(torch.load(load_pretrained_weights))

    offset = 0
    total = 0
    with torch.no_grad():
        for data in testLoader:
            #print(data)
            outputs = model(data[:,:16])
            #print(f'{outputs.shape}')
            for i in range(outputs.shape[0]):
                if i % 4000 == 3999:
                    print(f'Original value: {data[i,16:].item()}. \n Prediction: {outputs[i].item()}')
                offset += (outputs[i].item() * 100) / data[i, 16:].item()
            total += outputs.shape[0]
    writer.add_scalar('Test accuracy', offset / total)
    #print(f'The model is off by: {offset / total:.2f}%')
    with open(textFile, 'a') as f:
        f.write(f'The model is off by: {offset / total:.2f}%\n')

Converting the pandas dataframe to a torch tensor

In [5]:
def pdToTensor(dataframe, file=True):
    if not file:
        dataRefined = dg.filteredByHour(dataframe)
        ingredients = { 'tuna':'TunaWeight', 'meatball':'MeatballWeight', 'chicken':'ChickenWeight',
                'steak':'SteakWeight', 'chickenTeriyaki':'ChcknTkiWeight', 'cheese':'Cheese', 'tomato':'Tomato', 'olives':'Olives',
                'avocado':'Avocado'}
        finalDF = pd.DataFrame(columns=['Weather','Day','Hour','TypeOfFood','output'] )
        for row in dataRefined.iterrows():
            sample = {'Weather': row[1].Weather,'Day':row[0].dayofyear,'Hour':row[0].hour}
            for ing in ingredients:
                if row[1][ingredients[ing]] != 0:
                    sample['TypeOfFood'] = ing
                    sample['output'] = row[1][ingredients[ing]]
                    finalDF = finalDF.append(sample, ignore_index=True)
        finalDF = pd.concat([pd.get_dummies(finalDF['Weather'], prefix='', prefix_sep=''), finalDF], axis=1)
        finalDF = pd.concat([pd.get_dummies(finalDF['TypeOfFood'], prefix='', prefix_sep=''), finalDF], axis=1)
        del finalDF['Weather']
        del finalDF['TypeOfFood']
        finalDF.to_json('./data3.json')
        return torch.from_numpy(finalDF.values)
    newTensor = torch.from_numpy(dataframe.values)
    return newTensor

In [10]:
def main(args):
    ## TO DO: implement a tensorBoard to visualize the loss
    #writer = SummaryWriter()
    data1 = pd.read_json('./data1.json')
    data2 = pd.read_json('./data2.json')
    data3 = pd.read_json('./data3.json')

    data1 = pdToTensor(data1)
    data2 = pdToTensor(data2)
    data3 = pdToTensor(data3)
    ## Converting the dataframe to a tensor 
    #data4 = torch.cat([data1, torch.cat([data2, data3], dim=1)], dim=1)
    data = torch.cat((data1, torch.cat((data2, data3), 0)), 0)
    data = data.float()
    #print(data)
    #inputs (date/hour, type of ingredient)
    testSize = int(0.1 * len(data))
    trainSize = len(data)-testSize

    trainSet, testSet = torch.utils.data.random_split(data, [trainSize, testSize])


## Parameters to test

    trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=args['batch_size'], num_workers=0)
    testLoader = torch.utils.data.DataLoader(testSet, batch_size=args['batch_size'], num_workers=0)

#print(f"Training set consist of {len(trainSet)}, and the test set consists of {len(testSet)}")


    model = linearRegression(16,1)
    loss_fn = nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=args['lr'])
#print(model.parameters)
    train(model, opt, loss_fn, trainLoader, writer, n_epoch=args['epochs'], textFile=textFile)
    test(testLoader, model, writer, textFile)

In [11]:
if __name__ == '__main__':
    params = {'batch_size': 32, 'lr': 1e-2,'epochs':50}
    params = nni.get_next_parameter()
    main(params)